In [1]:
!pip freeze | grep scikit-learn

In [2]:
import pathlib

import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
year = 2022
month = 2

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
y_pred.std()

5.28140357655334

In [8]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df_result = pd.DataFrame()

In [10]:
df_result["ride_id"] = df["ride_id"]
df_result["predicted_duration"] = y_pred

In [11]:
output_dir = "output/yellow"
output_file = f"./{output_dir}/{year:04d}-{month:02d}.parquet"

pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

In [12]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [13]:
pathlib.Path(output_file).stat().st_size / 1024 / 1024

57.2156286239624